预测结果可视化

In [2]:
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from glob import glob
import json
import numpy as np
import shutil
from pathlib import Path
from tqdm import tqdm
import pickle

# jpg_json_path = "/media/ubuntu/Date12/TableStruct/data/test_A_jpg.json"
# jpg_info = json.load(open(jpg_json_path, 'r'))

def table_img_visualize(src_img_path, pred_json_path, dst_img_path):
    # 彩色检测框
    col2color = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                col2color.append((i*100, j*100, k*100))
    col2color = np.array(col2color)

    src_img = cv2.imread(src_img_path)
    json_data = json.load(open(pred_json_path, 'r'))
    img_id = os.path.basename(src_img_path).split(".")[0]
    # 遍历坐标列表
    for i, cell in enumerate(json_data["cells"]):
        x1, y1, x2, y2 = np.array(cell['bbox']).astype(np.int32).tolist()
        row_start, row_end = cell["row_start_idx"], cell["row_end_idx"]
        col_start, col_end = cell["col_start_idx"], cell["col_end_idx"]
        row_text = f"{row_start}" if row_start == row_end else f"{row_start}-{row_end}"
        col_text = f"{col_start}" if col_start == col_end else f"{col_start}-{col_end}"
        text = f"{row_text},{col_text}"

        color = col2color[row_start % col2color.shape[0]]
        cv2.rectangle(src_img, (x1, y1, x2-x1, y2-y1), color.tolist(), 1)

        width, height = src_img.shape[1], src_img.shape[0]
        font_scale = min(width, height)//1024
        font_scale = max(font_scale, 0.5)
        font_thickness = int(font_scale * 2)
        cv2.putText(src_img, text, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color.tolist(), font_thickness)

    cv2.imwrite(dst_img_path, src_img)


DATASET = "test_A_jpg480max_wireless"
PRED_PKL_DIR = f"../output/end2end_val_result"
img_data_dir    = f"/media/ubuntu/Date12/TableStruct/new_data/{DATASET}"
pred_visual_dir = f"./output/end2end_val_result/{DATASET}-detect_visualize/"
if os.path.exists(pred_visual_dir):
    shutil.rmtree(pred_visual_dir)
Path(pred_visual_dir).mkdir(parents=True, exist_ok=True)

pkls = sorted(glob(os.path.join(PRED_PKL_DIR, "detection_results_*.pkl")))

for result_file in pkls:
    with open(result_file, 'rb') as f:
        results = pickle.load(f)

    for result in tqdm(results):
        file_name = result['file_name']
        img_id = os.path.basename(file_name).split(".")[0]

        img_array = cv2.imread(os.path.join(img_data_dir, file_name))
        img_array = np.zeros_like(img_array)
        for bbox in result['bbox']:
            bbox = bbox.astype(np.int32).tolist()
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[4], bbox[5]
            # cv2.rectangle(img_array, (x1, y1, x2-x1, y2-y1), (0, 0, 255), 1)
            cv2.fillPoly(img_array, [np.array(bbox).reshape(-1, 2)], (255, 255, 255))
        dst_img_path = os.path.join(pred_visual_dir, file_name)
        cv2.imwrite(dst_img_path, img_array)



100%|██████████| 1410/1410 [00:05<00:00, 278.91it/s]
